In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../input/boston-crime-data/crime.csv', encoding='latin-1')
df.head()

In [ ]:
df = df.dropna(subset = ['Lat', 'Long'])
df = df[df['Lat'] > 0]

df

In [ ]:
df.info()


In [ ]:
df = df.dropna(subset=['UCR_PART'])
df

In [ ]:
df['OFFENSE_CODE_GROUP'].unique()

In [ ]:
important= ['Aggravated Assault', 
      'Harassment', 
       'Arson', 
       'Homicide', 
       'Criminal Harassment', 
       'Biological Threat',
       'Manslaughter', 'HUMAN TRAFFICKING',
        'Auto Theft', 'Larceny','Robbery','Residential Burglary','Larcency From Motor Vehicle','Other Burglary','Commercial Burglary'
   ]

# 'Aggravated Assault', 
#       'Harassment', 
#        'Arson', 
#        'Homicide', 
#        'Criminal Harassment', 
#        'Biological Threat',
#        'Manslaughter', 'HUMAN TRAFFICKING',
#         'Auto Theft', 'Larceny','Robbery','Residential Burglary','Larcency From Motor Vehicle','Other Burglary','Commercial Burglary']

In [ ]:
df['important']=0
i=0
for x in df['OFFENSE_CODE_GROUP']:
      if x in important:
        df['important'].iloc[i]=1
    
      i+=1
df

In [ ]:
df[df['important']==1]['OFFENSE_CODE_GROUP'].unique()

In [ ]:
df['DAY_OF_WEEK'] = df['DAY_OF_WEEK'].map({
    'Tuesday':4, 
    'Saturday':2, 
    'Monday':3, 
    'Sunday':1, 
    'Thursday':5, 
    'Wednesday':6,
    'Friday':7
})
df

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['REPORTING_AREA'] = encoder.fit_transform(df['REPORTING_AREA'])

In [ ]:
def get_dummies(dataframe):
    
#     for DISTRICT column
    
    DISTRICT = pd.get_dummies(dataframe.DISTRICT, prefix = 'DIS')
    dataframe = pd.concat([dataframe,DISTRICT],axis = 1)
    dataframe = dataframe.drop(['DISTRICT'],axis = 1)
    
    #for UCR PART column
    
    UCR_PART = pd.get_dummies(dataframe.UCR_PART, prefix = 'UCR')
    dataframe = pd.concat([dataframe,UCR_PART],axis = 1)
    dataframe = dataframe.drop(['UCR_PART'],axis = 1)
    
    return dataframe
df =get_dummies(df)
df

In [ ]:
df["OCCURRED_ON_DATE"]=pd.to_datetime(df["OCCURRED_ON_DATE"],infer_datetime_format=True)
df["OCCURRED_DAY"]=df["OCCURRED_ON_DATE"].apply(lambda x: x.date())
df = df.drop(['OCCURRED_ON_DATE'],axis = 1)
df

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df_2016 = df[df["YEAR"].isin([2016])]
df_2016

In [ ]:
validation_set2=df_2016.sample(30000)
validation_set2

In [ ]:
validation_set2[validation_set2['important']==1]

In [ ]:
validation_set2 = validation_set2[['REPORTING_AREA','MONTH','HOUR','DAY_OF_WEEK','DIS_A1','DIS_A15',
                'DIS_A7','DIS_B2','DIS_B3', 'DIS_C11', 'DIS_C6', 'DIS_D14', 'DIS_D4', 'DIS_E13', 'DIS_E18', 'DIS_E5','important']]
validation_set2

In [ ]:
groundTruth2 = validation_set2['important']
validation_set2 = validation_set2.drop('important', axis=1)
validation_set2

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
validation_set2_scaled = scaler.fit_transform(validation_set2)
validation_set2_scaled = pd.DataFrame(data=validation_set2_scaled, columns=['REPORTING_AREA','MONTH','HOUR','DAY_OF_WEEK','DIS_A1','DIS_A15',
                'DIS_A7','DIS_B2','DIS_B3', 'DIS_C11', 'DIS_C6', 'DIS_D14', 'DIS_D4', 'DIS_E13', 'DIS_E18', 'DIS_E5'])

validation_set2_scaled

In [ ]:
df = df[df["YEAR"].isin([2017,2018])]
df

In [ ]:
df.info()

In [ ]:
df[df.important==0]

In [ ]:
from sklearn.utils import resample

In [ ]:
df_majority = df[df.important==0]
df_minority = df[df.important==1]
 
# Upsample minority class
df_majority_downsampled  = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=135498,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_majority_downsampled])
 
# Display new class counts
df_upsampled.important.value_counts()


In [ ]:
df_upsampled = df_upsampled.sort_values(by =['OCCURRED_DAY'], ascending=False)
df_upsampled['OCCURRED_DAY']= pd.to_datetime(df_upsampled['OCCURRED_DAY'])
df_upsampled

In [ ]:
from sklearn.utils import shuffle
df_upsampled = shuffle(df_upsampled)

In [ ]:
df_upsampled

In [ ]:
train_set = df_upsampled[['REPORTING_AREA','MONTH','HOUR','DAY_OF_WEEK','DIS_A1','DIS_A15',
                'DIS_A7','DIS_B2','DIS_B3', 'DIS_C11', 'DIS_C6', 'DIS_D14', 'DIS_D4', 'DIS_E13', 'DIS_E18', 'DIS_E5','important']].head((int(len(df_upsampled) *0.7)))
train_set

In [ ]:
validation_set = df_upsampled[['REPORTING_AREA','MONTH','HOUR','DAY_OF_WEEK','DIS_A1','DIS_A15',
                'DIS_A7','DIS_B2','DIS_B3', 'DIS_C11', 'DIS_C6', 'DIS_D14', 'DIS_D4', 'DIS_E13', 'DIS_E18', 'DIS_E5','important']].tail(int(len(df_upsampled) *0.3))
validation_set

In [ ]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_set)
train_scaled = pd.DataFrame(data=train_scaled, columns=['REPORTING_AREA','MONTH','HOUR','DAY_OF_WEEK','DIS_A1','DIS_A15',
                'DIS_A7','DIS_B2','DIS_B3', 'DIS_C11', 'DIS_C6', 'DIS_D14', 'DIS_D4', 'DIS_E13', 'DIS_E18', 'DIS_E5','important'])

train_scaled

In [ ]:
groundTruth= validation_set['important'].to_frame()

groundTruth

In [ ]:
scaled_validation_set = scaler.fit_transform(validation_set)
scaled_validation_set = pd.DataFrame(data=scaled_validation_set, columns=['REPORTING_AREA','MONTH','HOUR', 'DAY_OF_WEEK', 'DIS_A1','DIS_A15',
                'DIS_A7','DIS_B2','DIS_B3', 'DIS_C11', 'DIS_C6', 'DIS_D14', 'DIS_D4', 'DIS_E13', 'DIS_E18', 'DIS_E5','important'])
scaled_validation_set = scaled_validation_set.drop('important', axis=1)
scaled_validation_set

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.semi_supervised import LabelSpreading
from sklearn.svm import LinearSVC
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def classification_metrics(y_test, predict):
    print("Confusion Matrix:\n")
    print(confusion_matrix(y_test, predict))
    print("\nAccuracy: ", accuracy_score(y_test, predict))
    print("\nClassification report:\n")
    print(classification_report(y_test, predict))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_scaled.drop(columns=["important"]), train_set["important"], random_state = 0, test_size = 0.33, shuffle=True) 

In [ ]:
from xgboost import XGBClassifier

xgB = XGBClassifier(learning_rate =0.1, n_estimators=1000,
    max_depth=10, min_child_weight=5, subsample=0.8,colsample_bytree=0.8,
    objective= 'binary:logistic',random_state=0)
xgB.fit(x_train, y_train)
xgb_test_predict = pd.DataFrame(data=xgB.predict(x_test))
classification_metrics(y_test, xgb_test_predict)

In [ ]:
xgb_valid_predict = pd.DataFrame(data=xgB.predict(scaled_validation_set))
xgB_accuracy = classification_metrics(groundTruth, xgb_valid_predict)
print('xgB_accuracy:', xgB_accuracy)

In [ ]:
xgb_valid2_predict = pd.DataFrame(data=xgB.predict(validation_set2_scaled))
xgB_accuracy = classification_metrics(groundTruth2, xgb_valid2_predict)
print('xgB_accuracy:', xgB_accuracy)


In [ ]:
lgbm = LGBMClassifier(random_state=0)
lgbm.fit(x_train, y_train)
lgbm_pred = lgbm.predict(x_test)

classification_metrics(y_test, lgbm_pred)

In [ ]:
predict_lgbm = pd.DataFrame(data=lgbm.predict(scaled_validation_set))
lgbm_accuracy = classification_metrics(groundTruth, predict_lgbm)
print('lgbm_accuracy:', lgbm_accuracy)

In [ ]:
predict_lgbm2 = pd.DataFrame(data=lgbm.predict(validation_set2_scaled))
lgbm_accuracy = classification_metrics(groundTruth2, predict_lgbm2)
print('lgbm_accuracy:', lgbm_accuracy)

In [ ]:
gauss = GaussianNB()
gauss = gauss.fit(x_train, y_train)
gauss_pred = gauss.predict(x_test)

classification_metrics(y_test, gauss_pred)

In [ ]:
predict = pd.DataFrame(data=gauss.predict(scaled_validation_set))
gauss_accuracy = classification_metrics(groundTruth, predict)
    
print('gauss_accuracy:', gauss_accuracy)

In [ ]:
predict2 = pd.DataFrame(data=gauss.predict(validation_set2_scaled))
gauss_accuracy = classification_metrics(groundTruth2, predict2)
    
print('gauss_accuracy:', gauss_accuracy)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbC = GradientBoostingClassifier(random_state=0, n_estimators=10)
gbC.fit(x_train, y_train)
predict = pd.DataFrame(data=gbC.predict(x_test), index = x_test.index)
classification_metrics(y_test, predict)

In [ ]:
predict = pd.DataFrame(data=gbC.predict(scaled_validation_set))
gbC_accuracy = classification_metrics(groundTruth, predict)
print('gbC_accuracy:', gbC_accuracy)

In [ ]:
predict2 = pd.DataFrame(data=gbC.predict(validation_set2_scaled))
gbC_accuracy = classification_metrics(groundTruth2, predict2)
print('gbC_accuracy:', gbC_accuracy)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abC = AdaBoostClassifier(random_state=0, n_estimators=10, learning_rate=0.9)
abC.fit(x_train, y_train)
predict = pd.DataFrame(data=abC.predict(x_test), index = x_test.index)
classification_metrics(y_test, predict)


In [ ]:
predict = pd.DataFrame(data=abC.predict(scaled_validation_set))
abC_accuracy = classification_metrics(groundTruth, predict)
print('abC_accuracy:', abC_accuracy)

In [ ]:
predict2 = pd.DataFrame(data=abC.predict(validation_set2_scaled))
abC_accuracy = classification_metrics(groundTruth2, predict2)
print('abC_accuracy:', abC_accuracy)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etC = ExtraTreesClassifier(random_state=0, n_estimators=50)

etC.fit(x_train, y_train)
predict = pd.DataFrame(data=etC.predict(x_test))
classification_metrics(y_test, predict)

In [ ]:
predict = pd.DataFrame(data=etC.predict(scaled_validation_set))
etC_accuracy = classification_metrics(groundTruth, predict)
print('etC_accuracy:', etC_accuracy)

In [ ]:
predict2 = pd.DataFrame(data=etC.predict(validation_set2_scaled))
etC_accuracy = classification_metrics(groundTruth2, predict2)
print('etC_accuracy:', etC_accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfC = RandomForestClassifier(criterion='entropy', max_depth= 8, max_leaf_nodes=5, min_samples_leaf=5, n_estimators= 100, random_state=0, class_weight='balanced_subsample')

rfC.fit(x_train, y_train)
predict = pd.DataFrame(data=rfC.predict(x_test))
classification_metrics(y_test, predict)

In [ ]:
predict = pd.DataFrame(data=rfC.predict(scaled_validation_set))
rfC_accuracy = classification_metrics(groundTruth, predict)
print('rfC_accuracy:', rfC_accuracy)

In [ ]:
predict2 = pd.DataFrame(data=rfC.predict(validation_set2_scaled))
rfC_accuracy = classification_metrics(groundTruth2, predict2)
print('rfC_accuracy:', rfC_accuracy)